In [3]:
import pandas as pd
import json

In [4]:

def load_json(file_path):
    '''
    Load json file
    '''
    with open(file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    return data

In [186]:
evidence_trafilatura = load_json("/Users/namle/DATN/MOOC/dataset/trafilatura_new.json")
raw_ris_results = load_json("/Users/namle/DATN/MOOC/dataset/ris_results.json")

In [185]:
raw_ris_results[0]

{'image_path': 'false_246.jpg',
 'urls': ['https://www.youtube.com/watch?v=HOl-zmrqqNM',
  'https://www.youtube.com/c/ThePorscheLover/videos',
  'https://www.wapcar.my/news/the-real-reason-why-the-great-shah-alam-flood-didn%E2%80%99t-affect-perodua-as-much-as-proton-40867',
  'https://shopee.com.my/Flood-Car-Bag-Plastik-Tebal-Panjang-Banjir-Kereta-Reusable-Thick-Waterproof-Plastic-Wrap-Protection-i.401638001.12663938248',
  'https://allianz.renewinsurance.my/news/increased-in-special-perils-take-up-rate/',
  'https://allianz.renewinsurance.my/category/news/',
  'https://allianz.renewinsurance.my/author/admin/',
  'https://www.lazada.com.my/products/flood-car-bag-plastik-tebal-panjang-banjir-kereta-reusable-thick-waterproof-plastic-water-flood-protection-anti-flood-car-wrap-plastik-bungkus-kereta-banjir-flood-protection-plastik-banjir-i2728587184.html',
  'https://allianz.renewinsurance.my/news/',
  'https://www.instagram.com/mazdausedcarplatform/',
  'https://www.thestar.com.my/news/na

In [187]:
def is_likely_html(url):
    '''
    Check that the evidence url is html
    '''
    # List of common file extensions
    file_extensions = ['.pdf', '.png', '.jpg', '.jpeg', '.gif', '.bmp', '.doc', '.docx', '.ppt', '.pptx', '.xls', 
                       '.xlsx', '.txt', '.zip', '.rar', '.exe', '.svg', '.mp4', '.avi', '.mp3']

    # Extract the extension from the URL
    extension = '.' + url.rsplit('.', 1)[-1].lower()

    # Check if the URL ends with a common file extension
    if extension in file_extensions:
        return False
    else:
        return True
def is_obfuscated_or_encoded(url):
    '''
    Check that the evidence url is not obfuscated or encoded.
    '''
    unquoted_url = url
    try:
        return '%' in unquoted_url or '//' in unquoted_url.split('/')[2]
    except:
        return True

In [ ]:
retrieval_results = []
for i in range(len(raw_ris_results)):
    for u in range(len(raw_ris_results[i]['urls'])):
      evidence_url = raw_ris_results[i]['urls'][u]
      ris_data = {
                'image_path': raw_ris_results[i]['image_path'],
                'raw_url': evidence_url,
                'image_urls': raw_ris_results[i]['image_urls'][evidence_url],
                'entities':raw_ris_results[i]['visual_entities'],
                'is_https': evidence_url.startswith('https')
            }
      ris_data['is_obfuscated'] = is_obfuscated_or_encoded(ris_data['raw_url'])  
      ris_data['is_html'] = is_likely_html(ris_data['raw_url'])
      ris_data['selection'] = ris_data['is_html'] and ris_data['is_https'] and not ris_data['is_obfuscated']
      retrieval_results.append(ris_data)
selected_retrieval_results = [d for d in retrieval_results if d['selection']]
# selected_retrieval_results = retrieval_results

In [235]:
len(raw_ris_results)

664

In [236]:
len(selected_retrieval_results)

9590

In [227]:
def get_item_by_image_path(data_list, image_path):

    for item in data_list:
        if item.get('image_path') == image_path:
            return item
    return None


In [229]:
t = get_item_by_image_path(raw_ris_results,"false_103.jpg")
t

{'image_path': 'false_103.jpg',
 'urls': [],
 'image_urls': {},
 'visual_entities': {'Bed': 0.45969998836517334,
  'Bedroom': 0.4189588129520416,
  'Sheet': 0.41630318760871887,
  'Henry Molaison': 0.359250009059906,
  'Stretcher': 0.33020704984664917,
  'Patient': 0.32102951407432556,
  'Clinic': 0.31738123297691345,
  'Clinic M Medical': 0.28859999775886536}}

In [226]:
raw_ris_results[1]

{'image_path': 'false_202.jpg',
 'urls': ['https://www.cgenomix.com/994f389.htm',
  'https://www.cnn.com/2019/08/08/us/rubber-duck-derby-trnd/index.html',
  'https://patricksdays.github.io/blog/chicago_patricks_day_river_syracuse_st_patricks_day_parade_2025_video.html',
  'https://www.cnn.com/videos/us/2019/08/08/63000-rubber-ducks-ducky-derby-chicago-ya-orig.cnn',
  'https://pizzaricetta.no/?p=265070416',
  'https://credcertosolucoes.com.br/?c=224569316',
  'https://asolar.fr/e1f82a4',
  'https://www.lumigen.com/?l=265070416',
  'https://landhaus-lemsdorf.de/?l=224569316',
  'https://cnnespanol.cnn.com/video/chicago-rio-patos-goma-inundan-es-viral-am-pkg-digital-original',
  'https://www.ukr.net/news/details/auto/99202354.html',
  'https://www.cnn.com/videos/spanish/2019/08/09/chicago-rio-patos-goma-inundan-es-viral-am-pkg-digital-original.cnn',
  'https://dianindahabadi.com/?d=224569316'],
 'image_urls': {'https://www.cgenomix.com/994f389.htm': ['https://media.cnn.com/api/v1/images/s

In [212]:
len(selected_retrieval_results)

10209

In [264]:
def clean_captions(series):
    cleaned = []
    for caption in series.dropna():
            if 'not found' not in caption.lower():
                cleaned.append(caption)
    return list(set(cleaned)) 
def merge_data(evidence, evidence_metadata, apply_filtering=False):
    '''
    Merge all evidence by dropping duplicates and applying 2 filters:
    1) The evidence is not the original FC article itself
    2) The evidence has been published before the FC article
    '''
    evidence_df = pd.DataFrame(evidence)
    evidence_metadata_df = pd.DataFrame(evidence_metadata)
    merged_data = pd.merge(evidence_df, evidence_metadata_df.drop_duplicates(subset='raw_url')[['image_path','raw_url']].rename(columns={'raw_url':'url'}), on='url',how='left')
    # merged_data  = merged_data.dropna(subset='evidence_url')
    filtered_df = merged_data[~(merged_data['description'].isnull() & merged_data['image'].isnull())]
    # merged_data = merged_data[['image_path','title','author',
    #                        'description', 'text','image','image_url','image_caption']]

    filtered_df = filtered_df[['image_path','title','author',
                           'description','image','image_url','image_caption']]
    # Step 2: Group by 'image_path' and aggregate
    merged_data = filtered_df.groupby('image_path').agg({
        'title': lambda x: list(set(x.dropna())),
        'author': lambda x: list(set(x.dropna())),
        'description': lambda x: list(set(x.dropna())),
        'image': lambda x: list(set(x.dropna())),
        'image_url': lambda x: list(set(x.dropna())),
        'image_caption': clean_captions
    }).reset_index()

    return merged_data

In [271]:
evidence_trafilatura[7]

{'title': '‘Do not start car during and after floods’',
 'author': 'CLEAN ADHESIVES',
 'url': 'https://www.thestar.com.my/news/nation/2021/12/21/do-not-start-car-during-and-after-floods',
 'description': 'GEORGE TOWN: Leave your car as it is in the event it gets trapped in a flood.',
 'text': 'Costly repairs: Several cars partly submerged in water outside a flat in Shah Alam. — Picture taken from social media\nGEORGE TOWN: Leave your car as it is in the event it gets trapped in a flood.\nTrying to start the engine after the floodwaters recede can cost you thousands of ringgit in repairs.',
 'image': 'https://apicms.thestar.com.my/uploads/images/2021/12/21/1414404.jpg',
 'image_url': 'https://apicms.thestar.com.my/uploads/images/2021/12/21/1414404.webp',
 'image_caption': 'Several areas hit by floods for the first time;Image not found'}

In [265]:
evidence = merge_data(evidence_trafilatura, selected_retrieval_results).fillna('').to_dict(orient='records')

In [266]:
len(evidence)

589

In [256]:
evidence[5]

{'image_path': 'false_102.jpg',
 'title': ['Zapotec civilization - Wikipedia',
  'The 5 Most Famous Civilisations of Pre-Columbian Art | Barnebys Magazine',
  'Zapotec Civilization',
  'Batlove Redux: Urgent Praise for our Winged Kin',
  'Mask of the Bat God',
  'The 5 Most Famous Civilizations of Pre-Columbian Art | Barnebys Magazine',
  'Zapotec Jade Bat Assemblage',
  'Zapotec Priest Figure',
  'Jade in Mesoamerica',
  'Archaeology & Art on Instagram: "The Jade Mask of the Zapotec Bat God, unearthed by archaeologists in the ancient city ruins of Monte Alban, symbolizes the rich cultural and religious tapestry of the powerful Zapotec civilization that once flourished between 100 BC and 200 AD. This remarkable artifact, crafted from 25 pieces of highly valued jade and featuring shell fragment eyes, is on display at the National Museum of Anthropology in Mexico City. It embodies the bat god, an important figure in the Zapotec and wider Mesoamerican belief systems due to bats\' associat

In [209]:
len(evidence)

589

In [257]:
import re

# Your data list of dictionaries
# Assume: data = [ {...}, {...}, ...]

# Function to extract number from image_path
def extract_number_from_path(item):
    match = re.search(r'_(\d+)', item['image_path'])
    return int(match.group(1)) if match else -1


In [202]:
# Assuming your data is in a list called `data`

false_evidence = [item for item in evidence if 'false' in item['image_path']]
true_evidence  = [item for item in evidence if 'true' in item['image_path']]


In [247]:
with open('ooc_new_evidence).json', 'w', encoding='utf-8') as f:
    json.dump(false_evidence, f, ensure_ascii=False, indent=2)

In [217]:
false_group = sorted(
    [item for item in evidence if 'false' in item['image_path']],
    key=extract_number_from_path
)

true_group = sorted(
    [item for item in evidence if 'true' in item['image_path']],
    key=extract_number_from_path
)


In [258]:
len(true_group)

315

In [259]:
len(false_group)

274

In [261]:
# Step 1: Generate expected list
expected = [f"true_{i}.jpg" for i in range(337)]  # 0 to 336

# Step 2: Extract actual image paths from false_group
present = [item['image_path'] for item in true_group]

# Step 3: Find missing samples
missing = sorted(list(set(expected) - set(present)))

print("Missings false_#.jpg samples:")
print(missing)


Missings false_#.jpg samples:
['true_1.jpg', 'true_103.jpg', 'true_117.jpg', 'true_125.jpg', 'true_133.jpg', 'true_137.jpg', 'true_148.jpg', 'true_159.jpg', 'true_176.jpg', 'true_186.jpg', 'true_206.jpg', 'true_231.jpg', 'true_240.jpg', 'true_245.jpg', 'true_263.jpg', 'true_280.jpg', 'true_310.jpg', 'true_311.jpg', 'true_325.jpg', 'true_46.jpg', 'true_7.jpg', 'true_75.jpg', 'true_95.jpg']


In [251]:
t = get_item_by_image_path(raw_ris_results,"true_1.jpg")
t

{'image_path': 'true_1.jpg',
 'urls': ['https://www.snopes.com/fact-check/electric-scooter-bike-abandoned-graveyard/',
  'https://www.snopes.com/tag/electric-vehicles/',
  'https://www.snopes.com/category/technology/?pagenum=9',
  'https://www.snopes.com/tag/editors-picks/?pagenum=7',
  'https://www.youtube.com/watch?v=wOsXYaI5JoI',
  'https://de.linkedin.com/in/ulrich-haeger-940209172',
  'https://de.linkedin.com/in/alexander-schmid-01a068b2',
  'https://de.linkedin.com/in/andreas-dengler-121a9b139',
  'https://factuel.afp.com/doc.afp.com.32ZR6BW',
  'https://www.liputan6.com/cek-fakta/read/5164327/cek-fakta-tidak-benar-foto-barisan-motor-listrik-mangkrak-karena-mahalnya-baterai',
  'https://correctiv.org/faktencheck/2022/12/07/keine-belege-dass-hunderte-elektromopeds-in-china-wegen-abgenutzter-batterien-aufgegeben-wurden/',
  'https://correctiv.org/faktencheck/wirtschaft-und-umwelt/',
  'https://de.linkedin.com/posts/lioudmila-thalmann_irgendetwas-stimmt-mit-unserem-umweltverstaendni

In [203]:
len(false_evidence)

274

In [208]:
false_evidence[4]

{'image_path': 'false_101.jpg',
 'title': ['Big rise in numbers of migrant children on Mexico-US border',
  "Border apprehensions up 71% in March from February's numbers",
  'Boulder supermarket shooting leaves 10 dead | First Thing',
  'Joe Biden raises Trump refugee cap after backlash',
  'immigrant detention',
  'Photos show crowded conditions for migrants at Texas Border Patrol facility',
  'Child migrants: Massive drop in children held by border officials',
  'HHS to open new holding facility as migrant presence, confusion on border grows',
  'migrant children',
  'Why thousands of immigrant children have spent more time than legally allowed at border',
  'Biden team was warned of potential border crisis, ex-border official says',
  'Migrants, including children, face cramped conditions at South Texas Border Patrol facility',
  'Sen. Chris Murphy steps carefully as border politics ripple north',
  "Key takeaways from President Biden's first news conference"],
 'author': ['Molly Bl

In [204]:
len(true_evidence)

315

In [78]:
raw_ris_results[0]

{'image_path': 'false_246.jpg',
 'urls': ['https://www.youtube.com/watch?v=HOl-zmrqqNM',
  'https://www.youtube.com/c/ThePorscheLover/videos',
  'https://www.wapcar.my/news/the-real-reason-why-the-great-shah-alam-flood-didn%E2%80%99t-affect-perodua-as-much-as-proton-40867',
  'https://shopee.com.my/Flood-Car-Bag-Plastik-Tebal-Panjang-Banjir-Kereta-Reusable-Thick-Waterproof-Plastic-Wrap-Protection-i.401638001.12663938248',
  'https://allianz.renewinsurance.my/news/increased-in-special-perils-take-up-rate/',
  'https://allianz.renewinsurance.my/category/news/',
  'https://allianz.renewinsurance.my/author/admin/',
  'https://www.lazada.com.my/products/flood-car-bag-plastik-tebal-panjang-banjir-kereta-reusable-thick-waterproof-plastic-water-flood-protection-anti-flood-car-wrap-plastik-bungkus-kereta-banjir-flood-protection-plastik-banjir-i2728587184.html',
  'https://allianz.renewinsurance.my/news/',
  'https://www.instagram.com/mazdausedcarplatform/',
  'https://www.thestar.com.my/news/na

In [76]:
evidence_trafilatura[0]

{'title': 'Update your browser',
 'author': None,
 'url': 'https://www.youtube.com/watch?v=HOl-zmrqqNM',
 'description': None,
 'text': '! [] (https://www.youtube.com/img/desktop/supported_browsers/dinosaur.png)\nUpdate your browser\nYour browser is no longer supported. Update your browser to get the most out of YouTube and our latest features. Learn moreRemember later\nYour browser is no longer supported. Update your browser to get the most out of YouTube and our latest features. Learn moreRemember later',
 'image': None,
 'image_url': 'https://i.ytimg.com/vi/HOl-zmrqqNM/hqdefault.jpg;https://i.ytimg.com/vi/HOl-zmrqqNM/mqdefault.jpg',
 'image_caption': 'Image not found;Image not found'}

In [81]:
evidence[0]

{'image_path': 'false_246.jpg',
 'title': 'Update your browser',
 'author': '',
 'description': '',
 'text': '! [] (https://www.youtube.com/img/desktop/supported_browsers/dinosaur.png)\nUpdate your browser\nYour browser is no longer supported. Update your browser to get the most out of YouTube and our latest features. Learn moreRemember later\nYour browser is no longer supported. Update your browser to get the most out of YouTube and our latest features. Learn moreRemember later',
 'image': '',
 'image_url': 'https://i.ytimg.com/vi/HOl-zmrqqNM/hqdefault.jpg;https://i.ytimg.com/vi/HOl-zmrqqNM/mqdefault.jpg',
 'image_caption': 'Image not found;Image not found'}

In [93]:
selected_retrieval_results[16]

{'image_path': 'false_246.jpg',
 'raw_url': 'https://loanstreet.com.my/learning-centre/list-of-flood-aids-to-apply-malaysia',
 'image_urls': ['https://cdn.loanstreet.com.my/learning_articles/images/000/001/272/original/loanstreet-list-of-flood-aids-to-apply-malaysia.jpg?1646815633'],
 'is_https': True,
 'is_obfuscated': False,
 'is_html': True,
 'selection': True}

In [ ]:
selected_retrieval_results = selected_retrieval_results[:16]
urls = [d['raw_url'] for d in selected_retrieval_results]
images = [d['image_urls'] for d in selected_retrieval_results]

In [ ]:
evidence_trafilatura

In [51]:
len(evidence_trafilatura)

312

In [57]:
len(selected_retrieval_results)

9590

In [21]:
evidence_trafilatura[0]

{'title': 'Update your browser',
 'author': None,
 'url': 'https://www.youtube.com/watch?v=HOl-zmrqqNM',
 'description': None,
 'text': '! [] (https://www.youtube.com/img/desktop/supported_browsers/dinosaur.png)\nUpdate your browser\nYour browser is no longer supported. Update your browser to get the most out of YouTube and our latest features. Learn moreRemember later\nYour browser is no longer supported. Update your browser to get the most out of YouTube and our latest features. Learn moreRemember later',
 'image': None,
 'image_url': 'https://i.ytimg.com/vi/HOl-zmrqqNM/hqdefault.jpg;https://i.ytimg.com/vi/HOl-zmrqqNM/mqdefault.jpg',
 'image_caption': 'Image not found;Image not found'}

In [117]:
import boto3
import requests
from trafilatura import bare_extraction, extract
import Levenshtein as lev
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

In [99]:
def compute_url_distance(url1,url2,threshold):
    distance = lev.distance(url1,url2)
    if distance < threshold:
        return True
    else:
        return False
def find_image_caption(soup, image_url,threshold=25):
    '''
    Retrieve the caption corresponding to an image url by searching the html in BeautifulSoup format.
    '''
    img_tag = None
    for img in soup.find_all('img'):
        src = img.get('src') or img.get('data-src') or img.get('data-original')
        if src and compute_url_distance(src, image_url, threshold):
            img_tag = img
            break
    if not img_tag:
        return "Image not found"
    figure = img_tag.find_parent('figure')
    if figure:
        figcaption = figure.find('figcaption')
        if figcaption:
            return figcaption.get_text().strip()
    for sibling in img_tag.find_next_siblings(['div', 'p','small']):
        if sibling.get_text().strip():
            return sibling.get_text().strip()
    title = img_tag.get('title')
    if title:
        return title.strip()
    # Strategy 4: Use the alt attribute of the image
    alt_text = img_tag.get('alt')
    if alt_text:
        return alt_text.strip()

    return "Caption not found"

In [ ]:
def merge_data(evidence, evidence_metadata, apply_filtering=False):
    '''
    Merge all evidence by dropping duplicates and applying 2 filters:
    1) The evidence is not the original FC article itself
    2) The evidence has been published before the FC article
    '''
    evidence_df = pd.DataFrame(evidence)
    evidence_metadata_df = pd.DataFrame(evidence_metadata)
    merged_data = pd.merge(evidence_df, evidence_metadata_df.drop_duplicates(subset='raw_url')[['image_path','raw_url']].rename(columns={'raw_url':'url'}), on='url',how='left')
    # merged_data  = merged_data.dropna(subset='evidence_url')
    filtered_df = merged_data[~(merged_data['description'].isnull() & merged_data['image'].isnull())]
    merged_data = merged_data[['image_path','title','author',
                           'description', 'text','image','image_url','image_caption']]
    # Step 2: Group by 'image_path' and aggregate
    merged_data = filtered_df.groupby('image_path').agg({
        'title': lambda x: list(set(x.dropna())),
        'author': lambda x: list(set(x.dropna())),
        'description': lambda x: list(set(x.dropna())),
        'image': lambda x: list(set(x.dropna())),
        'image_url': lambda x: list(set(url for sublist in x.dropna() for url in sublist)),
        'image_caption': lambda x: list(set(cap for sublist in x.dropna() for cap in sublist))
    }).reset_index()

    return merged_data

In [100]:
def extract_info_trafilatura(page_url,image_url):
    try:
        headers= {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(page_url, headers=headers, timeout=(10,10))
        if response.status_code == 200:
            #Extract content with Trafilatura
            result = bare_extraction(response.text,
                                   include_images=True,
                                   include_tables=False)
            #Remove unnecessary content
            keys_to_keep = ['title','author','url',
                            'description',
                            'text','image']
            result = {key: result[key] for key in keys_to_keep if key in result}
            result['image_url'] = image_url
            # Finding the image caption
            image_caption = []
            soup = bs(response.text, 'html.parser')
            for img in image_url:
                image_caption.append(find_image_caption(soup, img))
            if result['image']:
              image_caption.append(find_image_caption(soup,result['image']))
            result['image_caption'] = image_caption
            result['url'] = page_url
            return result
        else:
            return "Failed to retrieve webpage"
    except Exception as e:
        return f"Error occurred: {e}"

In [107]:
selected_retrieval_results = selected_retrieval_results[:16]
urls = [d['raw_url'] for d in selected_retrieval_results]
images = [d['image_urls'] for d in selected_retrieval_results]

In [108]:
urls

['https://www.youtube.com/watch?v=HOl-zmrqqNM',
 'https://www.youtube.com/c/ThePorscheLover/videos',
 'https://shopee.com.my/Flood-Car-Bag-Plastik-Tebal-Panjang-Banjir-Kereta-Reusable-Thick-Waterproof-Plastic-Wrap-Protection-i.401638001.12663938248',
 'https://allianz.renewinsurance.my/news/increased-in-special-perils-take-up-rate/',
 'https://allianz.renewinsurance.my/category/news/',
 'https://allianz.renewinsurance.my/author/admin/',
 'https://www.lazada.com.my/products/flood-car-bag-plastik-tebal-panjang-banjir-kereta-reusable-thick-waterproof-plastic-water-flood-protection-anti-flood-car-wrap-plastik-bungkus-kereta-banjir-flood-protection-plastik-banjir-i2728587184.html',
 'https://allianz.renewinsurance.my/news/',
 'https://www.instagram.com/mazdausedcarplatform/',
 'https://www.thestar.com.my/news/nation/2021/12/21/do-not-start-car-during-and-after-floods',
 'https://news.nestia.com/detail_share/8044929?media_type=1',
 'https://www.youtube.com/watch?v=h4apn0ODxYU',
 'https://www

In [119]:
output

[{'title': 'Please update your browser',
  'author': None,
  'url': 'https://www.youtube.com/watch?v=HOl-zmrqqNM',
  'description': None,
  'text': '! [] ( https://www.youtube.com/img/desktop/supported_browsers/dinosaur.png)\nPlease update your browser\nYour browser is no longer supported. Please update your browser for the best experience and enjoy the latest features on YouTube. Learn moreRemind me later\nYour browser is no longer supported. Please update your browser for the best experience and enjoy the latest features on YouTube. Learn moreRemind me later',
  'image': None,
  'image_url': ['https://i.ytimg.com/vi/HOl-zmrqqNM/hqdefault.jpg',
   'https://i.ytimg.com/vi/HOl-zmrqqNM/mqdefault.jpg'],
  'image_caption': ['Image not found', 'Image not found']},
 {'title': 'The Porsche Lover',
  'author': None,
  'url': 'https://www.youtube.com/c/ThePorscheLover/videos',
  'description': "Hi! I'm Adrian, The Porsche Lover & on my channel, I will be sharing the passion for Porsches. I will

In [120]:
evidence = merge_data(output, selected_retrieval_results).fillna('').to_dict(orient='records')

In [153]:
evidence

[{'image_path': 'false_246.jpg',
  'title': ['‘Do not start car during and after floods’',
   'The Porsche Lover',
   'Increased in Special Perils Take Up Rate - Renew Allianz Insurance Online',
   'Mazda Used Car Platform -Anshin- (@mazdausedcarplatform) • Instagram photos and videos',
   'admin, Author at Renew Allianz Insurance Online',
   'Flood Car Bag / Plastik Tebal Panjang Banjir Kereta / Reusable Thick Waterproof Plastic Water Flood Protection / Anti Flood Car Wrap / Plastik Bungkus Kereta Banjir / Flood Protection / Plastik Banjir',
   'Floody car bag'],
  'author': ['Admin', 'CLEAN ADHESIVES', 'The Star Online'],
  'description': ['Floody car bag. Gefällt 26.616 Mal. Floody bag To prevent the flooding Save money Agent/consumers are welcome to enquiry Exclusive Malaysia Distributor � 60192100256',
   "Hi! I'm Adrian, The Porsche Lover & on my channel, I will be sharing the passion for Porsches. I will also be sharing on car tips ranging from maintenance, technical and even pr